In [2]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.optim.lr_scheduler import StepLR, MultiStepLR
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
from cnn_finetune import make_model
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt

print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
0
1
NVIDIA GeForce GTX 1070


## Load and prepare train and test sets

In [ ]:
DATASET_PATH = "fashion_dataset/"

In [3]:
# Train set

train_df = pd.read_csv(DATASET_PATH + "train.csv", error_bad_lines=False).drop(['Unnamed: 0'], axis=1)
train_df['image'] = train_df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
train_df.head(5)

/home/ewaisha/anaconda3/envs/cnn_finetune/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,3559,Men,Footwear,Shoes,Casual Shoes,Brown,Winter,2015.0,Casual,Skechers Men's Casual Brown Lifestyle Shoe,3559.jpg
1,15232,Men,Footwear,Shoes,Casual Shoes,Brown,Fall,2011.0,Casual,ADIDAS Originals Men Vespa PK LO Brown Casual ...,15232.jpg
2,32186,Men,Footwear,Shoes,Sports Shoes,White,Summer,2012.0,Sports,ADIDAS Men Sprint Speed White Sports Shoes,32186.jpg
3,55618,Men,Footwear,Shoes,Casual Shoes,Black,Summer,2012.0,Casual,Numero Uno Men Black Shoes,55618.jpg
4,6830,Men,Footwear,Shoes,Sports Shoes,Black,Summer,2011.0,Sports,Nike Men's Zoom Vomero Black Shoe,6830.jpg


In [6]:
# Test set

test_df = pd.read_csv(DATASET_PATH + "test.csv", error_bad_lines=False).drop(['Unnamed: 0'], axis=1)
test_df['image'] = test_df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
test_df.head(5)

/home/ewaisha/anaconda3/envs/cnn_finetune/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,4412,Men,Apparel,Topwear,Tshirts,Black,Summer,2017.0,Casual,Pink Floyd Black Original Dark Side Black T-shirt,4412.jpg
1,23306,Women,Apparel,Topwear,Kurtas,Blue,Fall,2011.0,Ethnic,W Women Blue Kurta,23306.jpg
2,33908,Men,Accessories,Watches,Watches,Black,Winter,2016.0,Casual,Citizen Men Black Dial Watch BI0950-51E,33908.jpg
3,7170,Men,Apparel,Topwear,Shirts,Red,Fall,2011.0,Casual,Scullers Men Scul White Red Shirt,7170.jpg
4,20153,Men,Apparel,Topwear,Shirts,Blue,Fall,2011.0,Casual,Wrangler Men Taylor Block Check Blue Shirt,20153.jpg


### Limit dataset size (for faster training just for a proof of concept)

In [4]:
print(len(train_df))
train_df = train_df.sample(n=5000)
print(len(train_df))

34727
5000


In [7]:
print(len(test_df))
test_df = test_df.sample(n=1000)
print(len(test_df))

8682
1000


In [23]:
num_classes = len(train_df['subCategory'].unique())
print("Number of classes:", num_classes)

Number of classes: 22


In [ ]:
# dictionaries to map text labels to numbers

cls_map = {x:i for i, x in enumerate(train_df['subCategory'].unique())}
inv_cls_map = {x:i for i, x in cls_map.items()}

In [10]:
class CustomImageDataset(Dataset):
    """
    This class implememts a custom dataloader for our dataset.
    
    root_dir: the root directory for the dataset
    df: the dataframe to read (either train df or test df)
    transform: the image transformations to be done to the returned image
    """
    
    def __init__(self, root_dir, df, transform=None):
        self.root_dir = root_dir
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        img_id = str(self.df.iloc[index, 0])
        img = Image.open(os.path.join(self.root_dir, "images", img_id+".jpg")).convert("RGB")
        # get the sub-category label and map it to a number
        category = torch.tensor(cls_map[self.df.iloc[index, 3]])

        if self.transform is not None:
            img = self.transform(img)

        return {'image': img, 'label': category}

In [11]:
def make_classifier(in_features, num_classes):
    """
    this is the classifier that will be trained on top of the frozen backbone layers
    """
    return nn.Sequential(
        nn.Linear(in_features, 1024),
        nn.ReLU(),
        nn.Dropout2d(0.5),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Dropout2d(0.5),
        nn.Linear(512, num_classes),
        nn.LogSoftmax(dim=1)
    )

In [12]:
# training function

def train(model, device, train_loader, optimizer, epoch, writer):
    model.train()
    train_loss = 0
    correct = 0

    print("Epoch", epoch)
    for i_batch, item in tqdm(enumerate(train_loader), total=len(train_loader)):
        images = item['image'].to(device)
        labels = item['label'].to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = F.nll_loss(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        train_loss += F.nll_loss(outputs, labels, reduction='sum').item()
        pred = outputs.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(labels.view_as(pred)).sum().item()

    train_loss /= len(train_loader.dataset)
    print('Train Loss: {:.6f} \tTrain Accuracy: {}'.format(train_loss, 100. * correct / len(train_loader.dataset)))

    writer.add_scalar('Loss/train', train_loss, epoch)

In [ ]:
# Evaluation function

current_loss = 5
def test(model, device, test_loader, epoch, writer, exp_name):
    model.eval()
    global current_loss
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for item in test_loader:
            images = item['image'].to(device)
            labels = item['label'].to(device)
            outputs = model(images)
            test_loss += F.nll_loss(outputs, labels, reduction='sum').item()  # sum up batch loss
            pred = outputs.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(labels.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('Test Loss: {:.4f} \tTest Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    if test_loss <= current_loss:
        current_loss = test_loss
        print("Saving model with lowest loss..\n")
        torch.save(model.state_dict(), "./weights/{}_{}_{}.pt".format(exp_name, epoch, int(test_loss*100)))

    writer.add_scalar('Loss/test', test_loss, epoch)
    writer.add_scalar('Accuracy/test', correct/len(test_loader.dataset), epoch)

In [13]:
# Training settings

batch_size = 8
epochs = 50
lr = 0.0001
gamma = 0.1
no_cuda = False
seed = 1
save_interval = 10
save_model = True
exp_name = "resnet50"

use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 8, 'pin_memory': True} if use_cuda else {}
print("Using: {}".format(device))


# Image transforms
transforms_train = transforms.Compose([transforms.Resize((224, 224)),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                            std=[0.229, 0.224, 0.225])])
transforms_test = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                           std=[0.229, 0.224, 0.225])])

# Dataloaders
train_data_set = CustomImageDataset("./fashion_dataset", train_df, transforms_train)
train_loader = DataLoader(train_data_set, batch_size=batch_size, shuffle=True, **kwargs)

test_data_set = CustomImageDataset("./fashion_dataset", test_df, transforms_test)
test_loader = DataLoader(test_data_set, batch_size=batch_size, shuffle=True, **kwargs)


# Model
model = make_model('resnet50', num_classes=num_classes, pretrained=True,
                   input_size=(224, 224), classifier_factory=make_classifier).cuda()

## freeze feature extraction layers
for param in model._features.parameters():
    param.requires_grad = False


optimizer = torch.optim.Adam((param for param in model.parameters() if param.requires_grad), lr=lr)
scheduler = MultiStepLR(optimizer, milestones=[10,20], gamma=gamma)
writer = SummaryWriter(filename_suffix = exp_name)

Using: cuda


In [14]:
for epoch in range(1, epochs+1):
    try:
        train(model, device, train_loader, optimizer, epoch, writer)
        test(model, device, test_loader, epoch, writer, exp_name)
        scheduler.step()

        if epoch%save_interval == 0:
            print("Saving Model...\n")
            torch.save(model.state_dict(), "./weights/{}_{}.pt".format(exp_name, epoch))

    except KeyboardInterrupt:
        print("Stopping early. Saving network...\n")
        torch.save(model.state_dict(), "./weights/{}_{}.pt".format(exp_name, epoch))
        break

Epoch 1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [02:30<00:00,  4.15it/s]

Train Loss: 3.190897 	Train Accuracy: 57.7


Test Loss: 3.4388 	Test Accuracy: 759/1000 (76%)

Saving model with lowest loss..

Epoch 2


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [01:52<00:00,  5.56it/s]

...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [01:00<00:00, 10.38it/s]

Train Loss: 0.617055 	Train Accuracy: 91.44


Test Loss: 0.8490 	Test Accuracy: 906/1000 (91%)

Epoch 34


 22%|███████████████████████████▌                                                                                                | 139/625 [00:14<00:49,  9.78it/s]


Stopping early. Saving network...

